# 데이터 확인


1.   Raw 데이터
2.   GT값 (휴리스틱라벨링)
3.   정규분포에 따른 데이터 필터링
4.   our 데이터 전처리 시스템


In [24]:
import pandas as pd
#1
#file_path = 'pipe_final.xlsx'
#2
#file_path = 'heuristic_data.xlsx'
#3
#file_path = 'filter_pipe_final_241007.xlsx'
#4
#file_path = 'updated_total.xlsx'
#5
file_path = 'updated_total_filter.xlsx'

data = pd.read_excel(file_path).dropna()
data

,제품군,외경/폭,두께,등록 길이,등록 중량,step1 최종 작업 시간,step2 최종 작업 시간,step3 최종 작업 시간,step4 최종 작업 시간,step5 최종 작업 시간,step6 최종 작업 시간,filter
0,23A012-01,610.0,23.83,6.0,2.067,9.7,10.2,10.6,4.2,14.0,22.3,0
1,23A012-02,914.0,12.70,11.8,3.332,8.9,9.6,8.9,8.9,11.0,13.9,0
2,23A012-03,914.0,12.70,11.5,3.247,8.5,9.5,7.7,7.2,3.5,12.5,0
3,23A012-04,914.0,12.70,6.1,1.722,8.9,9.2,10.6,10.5,11.4,20.5,0
4,23A029-19,914.0,34.00,11.0,8.117,10.7,11.5,10.4,10.3,16.5,21.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
121,23Y020-04,864.0,9.50,6.0,1.202,5.4,9.9,10.5,9.4,14.0,21.6,0
122,23Y022-01,762.0,9.50,6.0,1.058,6.5,10.2,8.1,11.4,20.7,17.9,0
123,23Y023-01,762.0,25.40,11.8,5.445,11.6,11.7,7.3,9.4,18.5,10.4,0
124,23Y023-02,914.0,30.18,11.8,7.762,13.2,12.8,8.4,8.1,8.0,18.0,0


## input(외경/폭,등록길이,두께)에 따른 소요시간 값 예측 모델링

- 예측 및 평가 MAE(mean absolute error)로 값 계산하고 feature에 대한 중요도를 판별해서 input 값으로 사용

- RandomForestRegressor 모델 이용

In [25]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance
import pandas as pd

# 1. 데이터 불러오고 빈 데이터 없애기
data = pd.read_excel(file_path).dropna()

#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]

# 3. 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 다중 출력 회귀 모델 학습 (랜덤 포레스트)
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

# 5. 예측 및 평가
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print(f'Step-wise Mean Absolute Error: {mae}')

# 6. 예측 결과 확인
print(f'Predicted Times for each step: {y_pred[:5]}')
# 6. Permutation Importance 계산
perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)

# 7. 피처 중요도 출력
feature_importances = pd.DataFrame(perm_importance.importances_mean, index=X.columns, columns=['Importance'])
print("각 입력 변수의 중요도:")
print(feature_importances)

Step-wise Mean Absolute Error: [1.70011296 1.05172714 1.52127844 1.94818929 4.53957262 3.72119405]
Predicted Times for each step: [[ 7.101      12.15457143  8.9058      9.74375    16.289      16.627     ]
 [13.11086667 11.44386667  9.63173333 11.07725     8.9935     16.533     ]
 [ 7.65010476 10.87667143  9.01468333  8.99386667 12.29675    14.81471667]
 [10.95067619 11.0853381  11.4566619  11.78969048  0.8423381  14.39584286]
 [13.8372     12.16308333  6.8954      7.7813     12.6595     17.6346    ]]
각 입력 변수의 중요도:
       Importance
외경/폭     0.140386
두께       0.282847
등록 길이    0.008192


In [26]:
# 데이터에 따른 맞춤 변수 고려
"""
4,5 데이터 기준
file_path = 'updated_total.xlsx'
data = pd.read_excel(file_path)

#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]
--------------------------------------------------------------------------
1,2,3 데이터기준
# 2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭(mm)', '두께(mm)', '등록 길이(m)','등록 중량(ton)']]
#X = data[['외경/폭', '등록 길이','두께']]
y = data[['step1 작업 시간(분)', 'step2 작업 시간(분)',
         'step3 작업 시간(분)', 'step4 작업 시간(분)',
         'step5 작업 시간(분)', 'step6 작업 시간(분)']]
"""

"\n4,5 데이터 기준\nfile_path = 'updated_total.xlsx'\ndata = pd.read_excel(file_path)\n\n#2. 입력 변수(X)와 출력 변수(y) 설정\nX = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]\ny = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',\n         'step3 최종 작업 시간', 'step4 최종 작업 시간',\n         'step5 최종 작업 시간', 'step6 최종 작업 시간']]\n--------------------------------------------------------------------------\n1,2,3 데이터기준\n# 2. 입력 변수(X)와 출력 변수(y) 설정\nX = data[['외경/폭(mm)', '두께(mm)', '등록 길이(m)','등록 중량(ton)']]\n#X = data[['외경/폭', '등록 길이','두께']]\ny = data[['step1 작업 시간(분)', 'step2 작업 시간(분)',\n         'step3 작업 시간(분)', 'step4 작업 시간(분)',\n         'step5 작업 시간(분)', 'step6 작업 시간(분)']]\n"

# MLP regression 모델 사용

In [27]:
#사용x
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

# 1. 데이터 불러오고 빈 데이터 없애기
data = pd.read_excel(file_path).dropna()


#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]
# 3. 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. MLP 회귀 모델 학습
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# 5. 모델 성능 평가
y_pred = mlp.predict(X_test)
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print(f"Step-wise Mean Absolute Error: {mae}")

# 6. Permutation Importance 계산
perm_importance = permutation_importance(mlp, X_test, y_test, n_repeats=10, random_state=42)

# 7. 피처 중요도 출력
feature_importances = pd.DataFrame(perm_importance.importances_mean, index=X.columns, columns=['Importance'])
print("각 입력 변수의 중요도:")
print(feature_importances)


Step-wise Mean Absolute Error: [15.27695945  5.70038037  4.28242795  2.61483122 15.75941816  4.30224834]
각 입력 변수의 중요도:
       Importance
외경/폭     0.065050
두께      -0.246131
등록 길이    0.070647


- 필터링 데이터에 대한 MLP regressor

In [28]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

# 1. 데이터 불러오기
data = pd.read_excel(file_path).dropna()
  # null값이 있는 행을 제거

#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]
# 3. 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. MLP 회귀 모델 학습
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# 5. 모델 성능 평가
y_pred = mlp.predict(X_test)
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print(f"Step-wise Mean Absolute Error: {mae}")

# 6. Permutation Importance 계산
perm_importance = permutation_importance(mlp, X_test, y_test, n_repeats=10, random_state=42)

# 7. 피처 중요도 출력
feature_importances = pd.DataFrame(perm_importance.importances_mean, index=X.columns, columns=['Importance'])
print("각 입력 변수의 중요도:")
print(feature_importances)


Step-wise Mean Absolute Error: [15.27695945  5.70038037  4.28242795  2.61483122 15.75941816  4.30224834]
각 입력 변수의 중요도:
       Importance
외경/폭     0.065050
두께      -0.246131
등록 길이    0.070647


In [29]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

# 1. 데이터 불러오기
data = pd.read_excel(file_path).dropna()
  # null값이 있는 행을 제거

#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]
# 3. 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. MLP 회귀 모델 학습
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# 5. 모델 성능 평가
y_pred = mlp.predict(X_test)
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print(f"Step-wise Mean Absolute Error: {mae}")

# 6. Permutation Importance 계산
perm_importance = permutation_importance(mlp, X_test, y_test, n_repeats=10, random_state=42)

# 7. 피처 중요도 출력
feature_importances = pd.DataFrame(perm_importance.importances_mean, index=X.columns, columns=['Importance'])
print("각 입력 변수의 중요도:")
print(feature_importances)


Step-wise Mean Absolute Error: [15.27695945  5.70038037  4.28242795  2.61483122 15.75941816  4.30224834]
각 입력 변수의 중요도:
       Importance
외경/폭     0.065050
두께      -0.246131
등록 길이    0.070647


# MLP layer를 쌓아 학습에 사용

In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 1. 데이터 불러오기 및 전처리
data = pd.read_excel(file_path).dropna()
  # null값이 있는 행을 제거


#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]

# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 특성 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 2. 신경망 모델 구축
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(6)  # 출력 노드 수는 예측하고자 하는 작업 시간의 수
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 3. 모델 학습
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_split=0, verbose=1)

# 4. 모델 평가
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print('\n신경망 모델 평가:')
print('테스트 세트의 MSE:', loss)
print('테스트 세트의 MAE:', mae)

# 각 스텝별 평가 지표 계산
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

for i, step in enumerate(['step1', 'step2', 'step3', 'step4', 'step5', 'step6']):
    print(f"\n{step} 작업 시간(분) 평가:")
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print('MSE:', mse)
    print('MAE:', mae)
    print('R² 점수:', r2)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 528.9370 - mae: 12.0060
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 527.1936 - mae: 11.9578 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 794.4841 - mae: 12.4748  
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 791.6998 - mae: 12.4066  
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 523.2440 - mae: 11.8218 
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 524.5997 - mae: 11.8779 
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 519.9056 - mae: 11.6944 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 787.5362 - mae: 12.2448  
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 384.8391 - mae: 11.3737 
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 782.8729 - mae: 12.1227  
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 511.9474 - mae: 11.3827 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 780.7538 - mae: 12.0529  
Epoch 13/100
3/3 ━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

step1 작업 시간(분) 평가:
MSE: 5.578814056926266
MAE: 1.5875578668382433
R² 점수: -0.4254901415951846

step2 작업 시간(분) 평가:
MSE: 4.409934512101896
MAE: 1.5476973639594185
R² 점수: -1.0662898695874334

step3 작업 시간(분) 평가:
MSE: 4.443637891047612
MAE: 1.7725357320573596
R² 점수: -0.24307222066760503

step4 작업 시간(분) 평가:
MSE: 6.178061379096203
MAE: 1.938811355166965
R² 점수: -0.3282979553718546

step5 작업 시간(분) 평가:
MSE: 31.250714447353644
MAE: 4.433286343680488
R² 점수: -0.0013401809533708509

step6 작업 시간(분) 평가:
MSE: 48.51374439219285
MAE: 4.994784492916532
R² 점수: -2.393301926313735


In [31]:
# 신경망 모델을 사용했을 경우 저장
model.save('my_model.h5')

# 신경망 모델 로드
from tensorflow.keras.models import load_model
loaded_model = load_model('my_model.h5')

TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}

만약 학습할 데이터가 부족한 경우 (샘플이 부족한 경우)

In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 1. 데이터 불러오기 및 전처리
data = pd.read_excel(file_path).dropna()
  # null값이 있는 행을 제거


#2. 입력 변수(X)와 출력 변수(y) 설정
X = data[['외경/폭', '두께', '등록 길이']]#,'등록 중량']]
y = data[['step1 최종 작업 시간', 'step2 최종 작업 시간',
         'step3 최종 작업 시간', 'step4 최종 작업 시간',
         'step5 최종 작업 시간', 'step6 최종 작업 시간']]

# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 특성 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 2. 신경망 모델 구축
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(6)  # 출력 노드 수는 예측하고자 하는 작업 시간의 수
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 3. 모델 학습
# Determine the number of training samples
num_train_samples = X_train.shape[0]

# Define minimum samples required for validation_split=0.1
min_samples_for_validation = 10  # Adjust this threshold as needed

if num_train_samples >= min_samples_for_validation:
    validation_split = 0.1
    print(f"Using validation_split={validation_split} with {num_train_samples} training samples.")
else:
    validation_split = 0
    print(f"Insufficient training samples ({num_train_samples}). Skipping validation split.")

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=validation_split,
    verbose=1
)

# 4. 모델 평가
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print('\n신경망 모델 평가:')
print('테스트 세트의 MSE:', loss)
print('테스트 세트의 MAE:', mae)

# 각 스텝별 평가 지표 계산
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

for i, step in enumerate(['step1', 'step2', 'step3', 'step4', 'step5', 'step6']):
    print(f"\n{step} 작업 시간(분) 평가:")
    mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print('MSE:', mse)
    print('MAE:', mae)
    print('R² 점수:', r2)


Using validation_split=0.1 with 68 training samples.
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 518.4780 - mae: 12.0048 - val_loss: 140.3217 - val_mae: 11.2355
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 517.0744 - mae: 11.9450 - val_loss: 139.6931 - val_mae: 11.2093
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 873.1252 - mae: 12.6513 - val_loss: 139.0870 - val_mae: 11.1841
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 513.8115 - mae: 11.8301 - val_loss: 138.4981 - val_mae: 11.1597
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 874.1264 - mae: 12.7002 - val_loss: 137.9179 - val_mae: 11.1358
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 870.7508 - mae: 12.5798 - val_loss: 137.3269 - val_mae: 11.1111
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 513.3864 - mae: 11.8328 - val_loss: 136.7200 - val_mae: 11.0853
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 870.9086 - mae: 12.6008 - val_loss: 136.0989 - val_mae: 11.0592
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

step1 작업 시간(분) 평가:
MSE: 6.127354538631849
MAE: 1.864412143495348
R² 점수: -0.5656523769660482

step2 작업 시간(분) 평가:
MSE: 4.728895843296792
MAE: 1.695178487565782
R² 점수: -1.215740290138918

step3 작업 시간(분) 평가:
MSE: 4.980975971927245
MAE: 1.8530816713968916
R² 점수: -0.3933882585234352

step4 작업 시간(분) 평가:
MSE: 5.767399991505477
MAE: 1.8058488368988037
R² 점수: -0.24000477600452208

step5 작업 시간(분) 평가:
MSE: 27.947131530702062
MAE: 4.135679642359416
R² 점수: 0.1045137290789

step6 작업 시간(분) 평가:
MSE: 54.86771994075427
MAE: 5.326006656222874
R² 점수: -2.8377318036363683
